In [1]:
import geopandas as gpd
import pandas as pd
import os 
import glob
import requests
import zipfile
donwload_url = "https://catastotn.tndigit.it/export_semestrale_VL_PUBB/IDR0020230701_TIPOCATSH_CCXXX.zip"
url_csv = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSPeLuWTTF1JhWOhhR_ZJmSLBJhMqcJ771xWUeNnuX2co7aV2k2UytMRWU3AZdgfP4gIsWZZHsmx3T7/pub?output=csv"

In [2]:
df = pd.read_csv(url_csv)
codici_catastali = df.codice_comune_catastale.unique()

In [3]:
gdflist = []
for codice in codici_catastali:
    codice = str(codice).zfill(3)
    url = donwload_url.replace("XXX",codice)
    response = requests.get(url)
    # Elenca i nomi dei file che vuoi estrarre
    files_to_extract = []
    suffix = "_vl_uniqueparcel_poly"
    suffix = "_vl_parcel_poly"
    files_to_extract.append(codice + suffix + ".prj")
    files_to_extract.append(codice + suffix + ".shp")
    files_to_extract.append(codice + suffix + ".shx")
    files_to_extract.append(codice + suffix + ".dbf")
    if response.status_code == 200:
        with open("file.zip", "wb") as file:
            file.write(response.content)
        with zipfile.ZipFile("file.zip", "r") as zip_ref:
            for file_name in zip_ref.namelist():
                if file_name in files_to_extract:
                    zip_ref.extract(file_name)
        gdf = gpd.read_file(codice + suffix + ".shp")
        crs = gdf.crs
        gdflist.append(gdf)
        os.remove("file.zip")
        for shp in files_to_extract:
            os.remove(shp)
parcels = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True), crs=crs)

In [8]:
parcels['prg2'].unique()

array([''], dtype=object)

In [25]:
parcels['catasto'] = ""
parcels['comune'] = "NO"
parcels['prg1'] = ""
parcels['prg2'] = ""
parcels['prg3'] = ""
parcels['prg4'] = ""
parcels['prg5'] = ""
parcels['aggiornamento'] = ""
parcels["ettari"] = ""

for idx, row in df.iterrows():
    codice_comune_catastale= row['codice_comune_catastale']
    codice_particella = row['codice_particella']  
    p = parcels[(parcels.PT_CODE == codice_particella) & (parcels.PT_CCAT == codice_comune_catastale)]  
    if p.shape[0] >0:
        parcels.at[p.index[0],"comune"] = row['comune_ammistrativo']
        parcels.at[p.index[0],"catasto"] = row['nome_comune_catastale']
        parcels.at[p.index[0],"prg1"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"prg2"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"prg3"] = row['commento']
        parcels.at[p.index[0],"prg4"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"prg5"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"prg6"] = row['destinazione_uso_1']
        parcels.at[p.index[0],"aggiornamento"] = row['data_ultimo_aggiornamento_dati']
        parcels.at[p.index[0],"ettari"] = round(p.geometry.area[p.index[0]]/1000,2)

In [27]:
usi_civici = parcels[parcels.comune != "NO"]
usi_civici=usi_civici.to_crs(epsg=4326)
usi_civici.to_file("usi_civici.geojson")

In [28]:
usi_civici

,PT_TYPE,PT_CODE,PT_CCAT,PT_FABB,PT_PUB,geometry,catasto,comune,prg1,prg2,prg3,prg4,prg5,aggiornamento,ettari,prg6
27,P,2613,2,N,YES,"POLYGON ((11.04540 45.77297, 11.04540 45.77297...",ALA,ROVERETO,bosco,bosco,NaN,bosco,bosco,1938,12185.68,bosco
112,P,2216,2,N,YES,"POLYGON ((11.01832 45.78477, 11.01822 45.78460...",ALA,ROVERETO,improduttivo,improduttivo,NaN,improduttivo,improduttivo,1938,0.48,improduttivo
150,P,2341,2,N,YES,"POLYGON ((11.02331 45.79079, 11.02329 45.79088...",ALA,ROVERETO,improduttivo,improduttivo,NaN,improduttivo,improduttivo,1938,2.34,improduttivo
221,P,2318,2,N,YES,"POLYGON ((11.02225 45.78923, 11.02241 45.78921...",ALA,ROVERETO,improduttivo,improduttivo,NaN,improduttivo,improduttivo,1938,1.44,improduttivo
269,P,2406,2,N,YES,"POLYGON ((11.02582 45.79368, 11.02593 45.79356...",ALA,ROVERETO,improduttivo,improduttivo,NaN,improduttivo,improduttivo,1938,0.41,improduttivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259983,P,1290,256,N,YES,"POLYGON ((11.06033 45.88699, 11.06029 45.88688...",NORIGLIO,ROVERETO,pascolo,pascolo,NaN,pascolo,pascolo,1941,0.56,pascolo
260049,P,3427,256,N,YES,"POLYGON ((11.08662 45.88655, 11.08664 45.88657...",NORIGLIO,ROVERETO,pascolo,pascolo,NaN,pascolo,pascolo,1941,0.05,pascolo
260480,P,1410,256,N,YES,"POLYGON ((11.06746 45.88647, 11.06746 45.88648...",NORIGLIO,ROVERETO,agricolo,agricolo,una volta sottofrazione della frazione toldi p...,agricolo,agricolo,1941,0.49,agricolo
260731,P,1391,256,N,YES,"POLYGON ((11.06447 45.88651, 11.06456 45.88648...",NORIGLIO,ROVERETO,residenziale completamento,residenziale completamento,una volta sottofrazione della frazione toldi p...,residenziale completamento,residenziale completamento,1941,0.08,residenziale completamento
